<a href="https://colab.research.google.com/github/ChiriKamau/soma/blob/main/notebooks/soma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


path = "/content/drive/MyDrive/Soma/Population_scounty_age_gender.csv"


import pandas as pd
df = pd.read_csv(path)

print(df.shape)
print(df.columns)
df.head(10)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(48208, 6)
Index(['ewcounty', 'sub-county', 'Age', 'Male', 'Female', 'Total'], dtype='object')


,ewcounty,sub-county,Age,Male,Female,Total
0,BARINGO,ALL,Total,336322,330428,666750
1,BARINGO,ALL,0,8303,7982,16285
2,BARINGO,ALL,1,8669,8417,17086
3,BARINGO,ALL,2,10226,9762,19988
4,BARINGO,ALL,3,10265,10086,20351
5,BARINGO,ALL,4,11045,10276,21321
6,BARINGO,ALL,0 - 4,48508,46523,95031
7,BARINGO,ALL,5,10701,10047,20748
8,BARINGO,ALL,6,10949,10422,21371
9,BARINGO,ALL,7,9595,9082,18677


In [3]:

df_clean = df[df["Age"].astype(str).str.isdigit() | (df["Age"] == "Total")]

df_clean = df_clean.reset_index(drop=True)


print("Original :", len(df))
print("Filtered:", len(df_clean))
df_clean.head(20)


Original rows: 48208
Filtered rows: 39586


,ewcounty,sub-county,Age,Male,Female,Total
0,BARINGO,ALL,Total,336322,330428,666750
1,BARINGO,ALL,0,8303,7982,16285
2,BARINGO,ALL,1,8669,8417,17086
3,BARINGO,ALL,2,10226,9762,19988
4,BARINGO,ALL,3,10265,10086,20351
5,BARINGO,ALL,4,11045,10276,21321
6,BARINGO,ALL,5,10701,10047,20748
7,BARINGO,ALL,6,10949,10422,21371
8,BARINGO,ALL,7,9595,9082,18677
9,BARINGO,ALL,8,10276,10027,20303
